In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [150]:
import cv2
import numpy as np
from utils import *
from extractor import Extractor

In [151]:
fe=Extractor()

In [139]:
def projected_map(kps,depth):
    for p in kps:
        u,v=map(lambda x: int(round(x)),p.pt)
        Z=depth[v,u]
        # print(u,v,Z)

    


In [158]:

def process_img(img,depth):
    matches=fe.extract(img)
    ransac(matches)
    # print(matches[:-1][0])
    
    # projected_map(matches,depth)
    for kp1,kp2 in matches:
        u1,v1=map(lambda x: int(round(x)),kp1)
        u2,v2=map(lambda x: int(round(x)),kp2)
        cv2.circle(img,(u1,v1),color=(0,255,0),radius=3)
        cv2.line(img,(u1,v1),(u2,v2),color=(255,0,0))
        disp(img,"RGB")




In [174]:
if __name__ == "__main__":
    
    depth_paths='../rgbd_dataset_freiburg1_xyz/depth.txt'
    dlist=data(depth_paths)

    rgb_paths='../rgbd_dataset_freiburg1_xyz/rgb.txt'
    ilist=data(rgb_paths)

    dataset_path='../rgbd_dataset_freiburg1_xyz/'
    

    for i in range(len(dlist)):

        frame=cv2.imread(dataset_path+ilist[i])
        depth=cv2.imread(dataset_path+dlist[i],0)
        # print(frame.shape,depth.shape)


        process_img(frame,depth)
        # print(frame)

        if frame is None:
            print("End of frame")
            break

        
        disp(depth,"Depth")
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()


        


[[199. 185.]
 [127. 357.]
 [301. 191.]
 [522.  35.]
 [204. 201.]
 [ 98. 183.]
 [193. 211.]
 [212. 418.]
 [140. 253.]
 [214. 194.]
 [199. 381.]
 [200. 389.]
 [273. 307.]
 [151. 412.]
 [ 95. 194.]
 [128. 350.]
 [206. 399.]
 [256. 208.]
 [286. 260.]
 [ 97. 214.]
 [133. 342.]
 [289.  94.]
 [135.  57.]
 [302. 186.]
 [129. 368.]
 [201. 410.]
 [116. 288.]
 [238. 186.]
 [262. 179.]
 [247. 174.]
 [285. 238.]
 [117. 272.]
 [250. 213.]
 [132. 252.]
 [112. 278.]
 [131. 246.]
 [235. 233.]
 [162. 221.]
 [102. 204.]
 [269. 185.]
 [277. 202.]
 [133. 312.]
 [ 76. 255.]
 [ 87. 199.]
 [490. 278.]
 [302. 267.]
 [139. 189.]
 [ 89. 291.]
 [138. 397.]
 [106. 183.]
 [ 81. 193.]
 [ 79. 182.]
 [267. 317.]
 [120. 326.]
 [ 80. 254.]
 [107. 359.]
 [195. 402.]
 [112. 387.]
 [178. 338.]
 [141. 307.]
 [138. 340.]
 [354. 129.]
 [104. 395.]
 [589.  71.]
 [ 86. 195.]
 [ 87. 254.]
 [100. 253.]
 [109. 401.]
 [172. 214.]
 [218. 421.]
 [ 86. 421.]
 [387. 313.]
 [ 47. 319.]
 [586. 414.]
 [345. 130.]
 [386. 310.]
 [104. 344.]

In [364]:
def transformation(kp1,kp2):
    
    
    mux=np.mean(kp1,axis=0)
    muy=np.mean(kp2,axis=0)
    outer_products=[]
    var_x_products=[]
    var_y_products=[]

    for i in range(kp1.shape[0]):

        outer_products.append(np.outer(kp2[i,:]-muy,kp1[i,:]-mux))
        var_x_products.append(np.dot(kp1[i,:]-mux,kp1[i,:]-mux))
        var_y_products.append(np.dot(kp2[i,:]-muy,kp2[i,:]-muy))

    outer_products=np.array(outer_products)
    var_x_products=np.array(var_x_products)
    var_y_products=np.array(var_y_products)

    cov=np.mean(outer_products,axis=0)
    var_x=np.mean(var_x_products,axis=0)
    var_y=np.mean(var_y_products,axis=0)

    U,d,VT=np.linalg.svd(cov)
    
    D=np.diag(d)
    
    S=np.eye(D.shape[0])
    

    # print(np.linalg.det(U)*np.linalg.det(VT))
    if (np.linalg.det(U)*np.linalg.det(VT))<0:
        S[-1,-1]=-1
    # if np.linalg.det(cov)<1:
    #     

    R=np.dot(U,np.dot(S,VT))
    c=(1/var_x)*(np.trace(D*S))
    t=muy-c*np.dot(R,mux)
    
    error=var_y-((np.trace(D*S))**2)/var_x

    params={"R": R,"t":t,"c":c}
    return (params,error)
    

    


In [365]:
def ransac(matches):
    coordinate_array=np.array(matches)
    kp1=coordinate_array[:,0,:]
    kp2=coordinate_array[:,1,:]
    # print(kp1,kp2)
    params,error=transformation(kp1,kp2)
    # print(error)
    return(params)
    


In [366]:


matches = [((1, 2), (3, 4)), ((5, 6), (7, 8)), ((9, 10), (11, 12))]
ransac(matches)

{'R': array([[1.00000000e+00, 1.21168839e-16],
        [1.21168839e-16, 1.00000000e+00]]),
 't': array([2., 2.]),
 'c': 1.0}

In [371]:
th=np.pi*30/180.0

R1=np.array([[np.cos(th),-np.sin(th),0],
            [np.sin(th),np.cos(th),0],
            [0,0,1]])

c1=0.5
t1=np.array([2.5,0,3.0])
kp1=np.array([[1,2,3],[5,6,7],[9,10,11]])

matches=[]
for i in range(kp1.shape[0]):
    k1=kp1[i,:]
    k1 = tuple(k1)
    k2=c1*np.dot(R1,kp1[i,:])+t1
    k2 = tuple(k2)
    matches.append((k1,k2))

print(matches)
params=ransac(matches)
# print(params['R'])



[((1, 2, 3), (2.4330127018922196, 1.1160254037844386, 4.5)), ((5, 6, 7), (3.1650635094610973, 3.848076211353316, 6.5)), ((9, 10, 11), (3.897114317029975, 6.580127018922194, 8.5))]


In [372]:
R2=params['R']
t2=params['t']
c2=params['c']


matches2=[]

for i in range(kp1.shape[0]):
    k1=kp1[i,:]
    k1 = tuple(k1)
    k2=c2*np.dot(R2,kp1[i,:])+t2
    k2 = tuple(k2)
    matches2.append((k1,k2))
print(matches2)

[((1, 2, 3), (2.4330127018922214, 1.1160254037844386, 4.5)), ((5, 6, 7), (3.1650635094610973, 3.848076211353316, 6.5)), ((9, 10, 11), (3.897114317029973, 6.580127018922194, 8.5))]
